In [2]:
import requests
import json
from langchain_openai import ChatOpenAI
import os
from pydantic import BaseModel
from typing import List, Dict, Any
from langchain_core.prompts import ChatPromptTemplate
import os
import pathlib
from bs4 import BeautifulSoup
import re
from bs4 import BeautifulSoup
import re
from langchain_openai import ChatOpenAI
from pydantic import Field
from typing import Literal
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import HumanMessage, AIMessage
from __future__ import print_function
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from IPython.display import Markdown, display

In [14]:
with open(".env", "r") as f:
  for line in f:
    key, value = line.strip().split("=")
    os.environ[key] = value


In [4]:
search_terms = ["Software Engineer New Grad", "Entry level Software Engineer", "University Graduate Software Engineer", "Junior Software Developer", "AI automation Engineer"]

In [ ]:
class ResultRelevance(BaseModel):
  explanation: str
  id: str

class RelevanceCheckOutput(BaseModel):
  relevant_results: List[ResultRelevance]

def search_serper(search_query):
  url = "https://google.serper.dev/search"
    
  payload = json.dumps({
    "q": search_query,
    "gl": "us", 
    "num": 30,
    "tbs": "qdr:d"
  })

  headers = {
    'X-API-KEY': '0130e7da8283ec8a8d9721c6c38a97d557867923',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  results = json.loads(response.text)
  results_list = results['organic']

  all_results = []
  for id, result in enumerate(results_list, 1):
    result_dict = {
      'title': result['title'],
      'link': result['link'],
      'snippet': result['snippet'],
      'search_term': search_query,
      'id': id
    }
    all_results.append(result_dict)
  return all_results

def load_prompt(prompt_name):
  with open(f"prompts/{prompt_name}.md", "r") as file:
    return file.read()
  
def check_search_relevance(search_results: Dict[str, Any]) -> RelevanceCheckOutput:
  prompt = load_prompt("relevance_check")
  prompt_template = ChatPromptTemplate.from_messages(
    ("system_prompt", prompt)
  )

  llm = ChatOpenAI(model="gpt-4o").with_structured_output(RelevanceCheckOutput)
  llm_chain = prompt_template | llm
  return llm_chain.invoke({'input_search_results': search_results})

In [22]:
relevant_results = []
for search_term in search_terms:
  python_results = search_serper(search_term)
  results = check_search_relevance(python_results)
  
  # Get the relevant result IDs from the LLM output
  relevant_ids = [r.id for r in results.relevant_results]
  
  # Filter original results to only include those with matching IDs
  filtered_results = [r for r in python_results if str(r['id']) in relevant_ids]
  
  relevant_results.extend(filtered_results)
relevant_results

[{'title': '2025 New Grad Software Jobs (NOW HIRING) - ZipRecruiter',
  'link': 'https://www.ziprecruiter.com/Jobs/2025-New-Grad-Software',
  'snippet': 'Browse 39644 2025 NEW GRAD SOFTWARE jobs ($123k-$165k) from companies with openings that are hiring now. Find job postings near you and 1-click apply!',
  'search_term': 'Software Engineer New Grad',
  'id': 2},
 {'title': 'Entry Level Software Engineer Jobs | Robert Half',
  'link': 'https://www.roberthalf.com/us/en/jobs/all/entry-level-software-engineer',
  'snippet': 'Search and apply to our open Entry Level Software Engineer jobs. Our full-time, freelance and temporary Entry Level Software Engineer ...',
  'search_term': 'Software Engineer New Grad',
  'id': 3},
 {'title': 'Entry Level Software Developer Jobs in the USA (332 open now!)',
  'link': 'https://prosple.com/entry-level-software-developer-jobs',
  'snippet': 'Palantir Technologies USA. Software Engineer, New Grad - Infrastructure (Start ASAP). New York (Hybrid). USD 135k

In [23]:
def convert_html_to_markdown(html_content):
    # Create BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Convert common HTML elements to markdown
    
    # Headers
    for h in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        level = int(h.name[1])
        h.replace_with('#' * level + ' ' + h.get_text() + '\n\n')
    
    # Links
    for a in soup.find_all('a'):
        href = a.get('href', '')
        text = a.get_text()
        if href and text:
            a.replace_with(f'[{text}]({href})')
    
    # Bold
    for b in soup.find_all(['b', 'strong']):
        b.replace_with(f'**{b.get_text()}**')
    
    # Italic
    for i in soup.find_all(['i', 'em']):
        i.replace_with(f'*{i.get_text()}*')
    
    # Lists
    for ul in soup.find_all('ul'):
        for li in ul.find_all('li'):
            li.replace_with(f'- {li.get_text()}\n')
    
    for ol in soup.find_all('ol'):
        for i, li in enumerate(ol.find_all('li'), 1):
            li.replace_with(f'{i}. {li.get_text()}\n')
    
    # Get text and clean up
    text = soup.get_text()
    
    # Remove excess whitespace/newlines
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = text.strip()
    
    return text

def scrape_and_save_markdown(relevant_results):
    """
    Scrapes HTML content from URLs in relevant_results and saves as markdown files.
    
    Args:
        relevant_results: List of dictionaries containing search results with URLs
        
    Returns:
        List of dictionaries containing markdown content and metadata
    """
    # Create scraped_html directory if it doesn't exist
    pathlib.Path("scraped_markdown").mkdir(exist_ok=True)

    markdown_contents = []
    for result in relevant_results:
        if 'link' in result:
            payload = {
                "api_key": "zZwEqATYReQC4ogVNYnaLUYdm47ITPMNiFbRtvw1o7IpQbTubwZUvnkCycrWx2wCWt9wuA4aUIizsSvtQH", 
                "url": result['link'],
                "render_js": "true"
            }

            response = requests.get("https://scraping.narf.ai/api/v1/", params=payload)
            if response.status_code == 200:
                # Create filename from ID or URL if ID not available
                filename = f"{result.get('id', hash(result['link']))}.md"
                filepath = os.path.join("scraped_markdown", filename)
                
                # Convert HTML to markdown
                markdown_content = convert_html_to_markdown(response.content.decode())
                
                # Save markdown content to file
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(markdown_content)
                
                markdown_contents.append({
                    'url': result['link'],
                    'filepath': filepath,
                    'markdown': markdown_content,
                    'title': result.get('title', ''),
                    'id': result.get('id', '')
                })
            else:
                print(f"Failed to fetch {result['link']}: Status code {response.status_code}")

    print(f"Successfully downloaded and saved {len(markdown_contents)} pages as markdown to scraped_markdown/")
    return markdown_contents

markdown_contents = scrape_and_save_markdown(relevant_results)

Successfully downloaded and saved 54 pages as markdown to scraped_markdown/


In [26]:
markdown_contents

[{'url': 'https://www.ziprecruiter.com/Jobs/2025-New-Grad-Software',
  'filepath': 'scraped_markdown/2.md',
  'markdown': "2025 New Grad Software Jobs | ZipRecruiter Ireland\n\n[Skip to main content](#main)\n\n- \n[\n\nJob Seekers \n](#)\n\n[Log In](/profiles/sign_in)\n[Create your Profile](/profiles/sign_up)\n\n- \n[\n\nEmployers \n](#)\n\n[Log In](/employers/sign_in)\n[Create Employer Account](/employers/sign_up)\n\n[\n\nDistance\n\n](#)\n\n- [5 Kilometres](#)\n\n- [10 Kilometres](#)\n\n- [20 Kilometres](#)\n\n- [50 Kilometres](#)\n\n- [100 Kilometres](#)\n\n- [Any](#)\n\nSearch\n\n[ Back to results](#)\n### Refine your search\n\n##### Workplace\n\n- [On Site (10797)](/jobs/search?q=2025+New+Grad+Software&remote=on_site)\n\n### Get new jobs for this search by email\n\nIf you are a human, ignore this field\n\nCreate Alert\n\n    By clicking Create Alert, you agree to the [Global Terms of Use Agreement](https://www.ziprecruiter.global/en/terms) and acknowledge that you have read and un

In [ ]:
def generate_summaries(markdown_contents):
  pathlib.Path("markdown_summaries").mkdir(exist_ok=True)

  summary_prompt = load_prompt("summaries_markdown_page")

  summary_template = ChatPromptTemplate.from_messages([
    ("system", summary_prompt)
  ])

  llm = ChatOpenAI(model="gpt-4o")
  summary_chain = summary_template | llm

  summaries = []
  for content in markdown_contents:
    try:
      summary = summary_chain.invoke({
        'markdown_input': ' '.join(content['markdown'].split()[:2000])
      })
      
      # Create filename for summary
      summary_filename = f"summary_{content['id']}.md"
      summary_filepath = os.path.join("markdown_summaries", summary_filename)
      
      # Save summary to file
      with open(summary_filepath, 'w', encoding='utf-8') as f:
        f.write(summary.content)
      
      # Add to summaries list
      summaries.append({
        'markdown_summary': summary.content,
        'url': content['url']
      })

    except Exception as e:
      print(f"Failed to summarize {content['filepath']}:{str(e)}")
  
  print(f"Successfully generated summaries for {len(markdown_contents)} pages in markdown_summaries/")
  return summaries


summaries = generate_summaries(markdown_contents)


In [ ]:
summaries